In [6]:
import torch
import torch.nn as nn
import onnxruntime
import onnx
from PIL import Image
import numpy as np
import torchvision.transforms as transforms

In [2]:
onnxruntime.__version__

'1.8.1'

In [3]:
onnx_model = onnx.load("wf_scale2x_new.onnx")
onnx.checker.check_model(onnx_model)

In [4]:
session = onnxruntime.InferenceSession("wf_scale2x_new.onnx")

In [5]:
def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

In [10]:
img = Image.open('./FH3_1_540p.png').convert('RGB')


In [11]:
to_tensor = transforms.ToTensor()
img_rgb = to_tensor(img)
img_rgb.unsqueeze_(0)
img_rgb.shape

torch.Size([1, 3, 540, 960])

In [17]:
inputs_data = {session.get_inputs()[0].name : to_numpy(img_rgb)}
outputs = session.run(None, inputs_data)
outputs[0].shape

(1, 3, 1080, 1920)

In [22]:
img_out = outputs[0]
img_out = np.transpose(img_out[0], (1, 2, 0))
img_2x = Image.fromarray(np.uint8((img_out * 255.0).clip(0, 255)), mode='RGB')


In [23]:
img_2x.save('fh3_1_1080p_rgb.png')